# Getting the data

In [1]:
import preprocessing
import featureSelection
import time
def eval_metrics(y, y_predicted):
    from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
    print(accuracy_score(y, y_predicted))
    print(precision_score(y, y_predicted))
    print(recall_score(y, y_predicted))
    print(f1_score(y, y_predicted))

Selection based on Pearson Correlation


c:\users\aaate\appdata\local\programs\python\python38\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\aaate\appdata\local\programs\python\python38\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


30 selected features
['Bwd IAT Max', 'ACK Flag Count', 'Fwd Packet Length Min', 'URG Flag Count', 'Destination Port', 'Fwd IAT Mean', 'Bwd IAT Std', 'Flow IAT Mean', 'PSH Flag Count', 'FIN Flag Count', 'Flow Duration', 'Fwd IAT Total', 'Bwd Packet Length Min', 'Min Packet Length', 'Flow IAT Std', 'Idle Min', 'Fwd IAT Max', 'Flow IAT Max', 'Idle Mean', 'Idle Max', 'Average Packet Size', 'Packet Length Mean', 'Fwd IAT Std', 'Packet Length Variance', 'Max Packet Length', 'Packet Length Std', 'Avg Bwd Segment Size', 'Bwd Packet Length Mean', 'Bwd Packet Length Max', 'Bwd Packet Length Std']
Selection based on chi square distibution
30 selected features
['Destination Port', 'Flow Duration', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Std', 'Bwd IAT Max', 'Fwd PSH Flags', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance

In [2]:
corf = featureSelection.cor_feature[:23]
chif = featureSelection.chi_feature[:25]
# corfm = corf[:10] + corf[20:]
# print(len(corfm))

schi = set(corf)
scor = set(chif)

sint = schi.intersection(scor)
feature_list = list(sint)
print(len(feature_list))

16


In [3]:
X_train, y_train = featureSelection.X_train, featureSelection.y_train
X_train = X_train[feature_list]

In [4]:
X_cv, y_cv = featureSelection.X_cv, featureSelection.y_cv
X_cv = X_cv[feature_list]

In [5]:
X_train.info()
X_cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545088 entries, 0 to 2545087
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Destination Port     float64
 1   Bwd IAT Max          float64
 2   PSH Flag Count       float64
 3   Packet Length Mean   float64
 4   URG Flag Count       float64
 5   ACK Flag Count       float64
 6   Fwd IAT Mean         float64
 7   Fwd IAT Std          float64
 8   Bwd IAT Std          float64
 9   Fwd IAT Max          float64
 10  Fwd IAT Total        float64
 11  Flow IAT Max         float64
 12  Average Packet Size  float64
 13  FIN Flag Count       float64
 14  Flow IAT Std         float64
 15  Flow Duration        float64
dtypes: float64(16)
memory usage: 310.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141394 entries, 0 to 141393
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Destination Po

# Neural Network

In [ ]:
import tensorflow as tf
import tensorflow.keras
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import random
from numpy.random import seed
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam


# X_train = X_train.loc[:65535]
# y_train = y_train[:65536]
model = Sequential()
model.add(Dense(2*len(feature_list), input_dim=len(feature_list), activation='relu'))
model.add(Dense(2*len(feature_list), activation='relu'))
model.add(Dense(len(feature_list), activation='relu'))
# model.add(Dense(2*len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
model.add(Dense(len(feature_list)//2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.005), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=256)


In [ ]:
print('Train metrics')
t_train = time.time()
y_train_pred = model.predict_classes(X_train)
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = model.predict_classes(X_cv)
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

# Ensemble Learning

In [7]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

In [8]:
rdf_c = RandomForestClassifier(n_estimators=300,random_state=0)
knn_clf = KNeighborsClassifier()
ExtraTrees_clf = ExtraTreesClassifier(n_estimators=100)
gradient_boost = GradientBoostingClassifier()
ada_mod = AdaBoostClassifier(base_estimator= None)
bg_mod = BaggingClassifier(base_estimator= None, bootstrap_features=False)

In [ ]:
vot_classifier = VotingClassifier(estimators=[
    ('bagging_classifier', bg_mod),
    ('rnd_classifier', rdf_c),
    ('ExtraTreesClassifier', ExtraTrees_clf),
    ('KNN_classifier', knn_clf),
],voting='soft', n_jobs=-1)

vot_classifier.fit(X_train, y_train)

In [ ]:
print('Train metrics')
t_train = time.time()
y_train_pred = vot_classifier.predict(X_train)
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = vot_classifier.predict(X_cv)
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

In [ ]:
for clf in (ExtraTrees_clf, knn_clf, vot_classifier):
    clf.fit(X_train, y_train)
    y1 = clf.predict(X_train)
    print(clf.__class__.__name__, accuracy_score(y_train, y1))
    print(precision_score(y_train, y1))
    print(recall_score(y_train, y1))
    print(f1_score(y_train, y1))

In [ ]:
for clf in (ExtraTrees_clf, knn_clf, vot_classifier):
    clf.fit(X_cv, y_cv)
    y1 = clf.predict(X_cv)
    print(clf.__class__.__name__, accuracy_score(y_cv, y1))
    print(precision_score(y_cv, y1))
    print(recall_score(y_cv, y1))
    print(f1_score(y_cv, y1))

# ExtraTrees

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
ExtraTrees_clf = ExtraTreesClassifier(n_estimators=100)
ExtraTrees_clf.fit(X_train, y_train)

ExtraTreesClassifier()

In [8]:
print('Train metrics')
t_train = time.time()
y_train_pred = ExtraTrees_clf.predict(X_train)
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = ExtraTrees_clf.predict(X_cv)
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

Train metrics
Pred time on train set:  41.313191652297974
0.9980912251364197
0.9942882652349464
0.996023158315033
0.9951549556485957
CV metrics
Pred time on cv set:  2.3473451137542725
0.9972841846188665
0.9923220436280138
0.993891044990657
0.9931059245960503


In [11]:
import joblib 
joblib.dump(ExtraTrees_clf, 'extratrees.sav') 



['extratrees.sav']

In [13]:
import joblib 
model = joblib.load('extratrees.sav')
y_cv_pred = model.predict(X_cv)
eval_metrics(y_cv, y_cv_pred)


0.9972841846188665
0.9923220436280138
0.993891044990657
0.9931059245960503
